# Veeam API and Python Part 5 - Exploring the API aka how to work with response data

## Updated with the new version of Veeam Easy Connect

In this notebook we will go over some of the methods that I work with the data you get back from the API.

First the imports:

In [2]:
from veeam_easy_connect import VeeamEasyConnect
import getpass

Next log into the v11 API using Veeam Easy Connect

In [4]:
username = "username"
password = getpass.getpass("Enter password: ")
address = "YOUR_VBR_API_ADDRESS" # ip or DNS

veeam_ec = VeeamEasyConnect(username, password, False) # Insecure
header = veeam_ec.vbr().login(address)

Grab an endpoint from the Veeam Documentation, in this case we will use the /api/v1/jobs endpoint.

In [6]:
url = f"https://{address}:9419/api/v1/jobs"

Then make a request to get the data

In [7]:
job_data = veeam_ec.get(url)

## Save the data for later use

Here is how you can save the data to a JSON file that can be read later.

In [62]:
import json
with open("job_data2.json", "w") as job_data_file:
    json.dump(job_data, job_data_file)

And here is how you can read the data back in.

In [ ]:
with open("job_data2.json", "r") as job_data_file:
    job_data = json.load(job_data_file)

## Data Structures

An example of a List (aka Array) and how to pull out a value using an index

In [63]:
my_list = [1,2,3,4,5,6,7,8]

In [65]:
my_list[1]

2

An Example of an Dictionary (aka object)

In [22]:
data = {
    "name": "ed", 
    "age": 32
}

How we can get one value by calling the key

In [23]:
data['name']

'ed'

And how we can get all the keys and values

In [3]:
print(data.keys())
print(data.values())

dict_keys(['name', 'age'])
dict_values(['ed', 32])


Here we are showing that we can create a List of Dictionaries (Array of Objects)

In [4]:
dict_list = [data, data, data, data]

In [5]:
dict_list

[{'name': 'ed', 'age': 32},
 {'name': 'ed', 'age': 32},
 {'name': 'ed', 'age': 32},
 {'name': 'ed', 'age': 32}]

And how we can first index out the first dictionary and then access a key in the dictionary

In [6]:
dict_list[0]['name']

'ed'

## /api/v1/jobs object/dict walk through

Here we will walk through the dictionary that we received back from the API to pull out relevant data.

The aim here is to be able to create a new list of dictionaries that have the:

* Job Name
* VMs in the Job

We start with checking for the keys in the response.

In [12]:
job_data.keys()

dict_keys(['data', 'pagination'])

Here we can see we have to keys, we want to follow the 'data' path

NOTE the ";" has been added to remove the output for GitHub, remove when running it yourself.

In [9]:
job_data['data'];

We can see from the [ that we are looking at a List so we need to index the first item out (job)

In [11]:
job_data['data'][0];

That removes the [ at the start so we know we are looking at a Dictionary.

Next we can have a look at the keys

In [12]:
job_data['data'][0].keys()

dict_keys(['isHighPriority', 'virtualMachines', 'storage', 'guestProcessing', 'schedule', 'type', 'id', 'name', 'description', 'isDisabled'])

Looking at the saved JSON file we know that the VMs are inside the "virtualMachines" and the Job name is under "name".

We'll aim for the Job name first

In [13]:
job_data['data'][0]['name']

'ActiveDirectory'

Next we need to follow the same process as before, following data structure to the data we need.

In [14]:
job_data['data'][0]['virtualMachines']['includes'][0]['inventoryObject']['name']

'WS2016-AD-DNS'

Here we will use a double loop to get the data from the API response that we want.

In [15]:
# Create a list to receive the new dicts
job_vm_map = []

# Start the first loop
for i in job_data['data']:
    # create a temporary list to hold all the VM names from each job
    vm_names = []
    # Start the second loop through the VM dicts
    for j in i['virtualMachines']['includes']:
        data = j['inventoryObject']['name']
        # Add them to the temporary List
        vm_names.append(data)
    # Create a new dict with the Job Name and the VMs
    job_info = {
        "jobName": i['name'],
        "vmsInJob": vm_names
    }
    # Add that to the first list
    job_vm_map.append(job_info)

Finally, we can now look at the new List of Dicts which have the Job Names and VMs mapped

In [16]:
job_vm_map

[{'jobName': 'ActiveDirectory', 'vmsInJob': ['WS2016-AD-DNS']},
 {'jobName': 'k8s',
  'vmsInJob': ['K8s-Control', 'K8s-Worker1', 'K8s-Worker2', 'K8s-Worker3']},
 {'jobName': 'Mixed SOBR', 'vmsInJob': ['Small Debian2']},
 {'jobName': 'PerJob', 'vmsInJob': ['Small Debian2', 'Small Debian3']},
 {'jobName': 'SQL1', 'vmsInJob': ['SQL1']},
 {'jobName': 'Tiny NFS vm', 'vmsInJob': ['TinyVM']},
 {'jobName': 'TinyVM2Netapp', 'vmsInJob': ['TinyVM']}]

Next video we will take a look at Pandas 🐼